# Parameter Fit as a Function of SNR - Code Description

This code serves to study how changing the SNR in data influences the nonlinear least squares (NLLS) estimation of parameter values

We are expecting to see an order of convergence for the estimated data to the real data for a model based on how much noise is added to the data

The principle contributor to this code is Griffin Hampton (GSH)